# CSE 151A Group Project: Tree Models

In [1]:
# All of our imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import uniform, poisson
import warnings

Run to suppress warnings when running code

In [2]:
warnings.filterwarnings('ignore')

First, we must get our processed data to train the model

In [3]:
input_df = pd.read_csv('processed_input.csv')
output_df = pd.read_csv('processed_output.csv')
display(input_df)
display(output_df)

,host_since,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,accommodates,bedrooms,price,minimum_nights,maximum_nights,...,Bathroom essentials,Baby safety gates,Bread maker,Clothing storage,Full kitchen,Fireplace guards,Sound system,EV charger,Outdoor furniture,Lake access
0,0.735956,0.0,0.000759,1.0,0.0,0.066667,0.0,0.002761,0.001002,0.013138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.577049,0.0,0.000759,1.0,1.0,0.066667,0.0,0.006653,0.001002,0.013138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.523716,0.0,0.000759,1.0,0.0,0.066667,0.0,0.004852,0.001002,0.013138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.573115,0.0,0.000759,1.0,1.0,0.066667,0.0,0.003052,0.001002,0.013138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.493989,0.0,0.000759,1.0,0.0,0.066667,0.0,0.003168,0.001002,0.013138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168318,0.467760,0.0,0.000759,1.0,1.0,0.066667,0.0,0.006653,0.000000,0.000070,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
168319,0.577486,0.0,0.000759,1.0,1.0,0.066667,0.0,0.003168,0.006012,0.000164,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
168320,0.704044,0.0,0.000759,1.0,1.0,0.066667,0.0,0.002587,0.005010,0.000339,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
168321,0.447213,0.0,0.000759,1.0,1.0,0.066667,0.0,0.005782,0.002004,0.000199,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,review_scores_rating
0,100.0
1,100.0
2,100.0
3,100.0
4,100.0
...,...
168318,100.0
168319,100.0
168320,100.0
168321,100.0


In [4]:
X_train, X_test, y_train, y_test = train_test_split(input_df,output_df, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=0.2, random_state=42)

print(f'Training data shapes: {X_train.shape}, {y_train.shape}')
print(f'Validation data shapes: {X_val.shape}, {y_val.shape}')
print(f'Test data shape: {X_test.shape}, {y_test.shape}')

Training data shapes: (107726, 247), (107726, 1)
Validation data shapes: (26932, 247), (26932, 1)
Test data shape: (33665, 247), (33665, 1)


In [5]:
validation_error_list = []
training_error_list = []
testing_error_list = []

In [6]:
decision_tree_model=DecisionTreeRegressor()
decision_tree_model.fit(X_train,y_train)

DecisionTreeRegressor()

In [7]:
# Show our starting off point for error
yhat_train = decision_tree_model.predict(X_train)
train_error = mean_squared_error(y_train,yhat_train)
training_error_list.append(np.log(train_error))
print('Training Error:',train_error)

yhat_test = decision_tree_model.predict(X_test)
test_error = mean_squared_error(y_test,yhat_test)
testing_error_list.append(np.log(test_error))
print('Testing Error:',test_error)

yhat_val = decision_tree_model.predict(X_val)
val_error = mean_squared_error(y_val,yhat_val)
validation_error_list.append(np.log(val_error))
print('Validation Error:',val_error)

Training Error: 0.7772087016686742
Testing Error: 188.78415467461062
Validation Error: 177.90688976487306


In [8]:
# parameters={"splitter":["best","random"],
#             "max_depth" : [1,3,5,7,9,11,12],
#            "min_samples_leaf":[1,2,3,4,5,6,7,8,9,10],
#            "min_weight_fraction_leaf":[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
#            "max_features":["auto","log2","sqrt",None],
#            "max_leaf_nodes":[None,10,20,30,40,50,60,70,80,90] }
# parameters = {
#     'splitter':["best","random"],
#     'max_depth': [3,5,7],
#     'min_samples_split': [3,5,7,9],
#     'min_samples_leaf': [3,5,7,9],
#     "max_leaf_nodes":[10,20,30,40]
# }
parameters = {'criterion':['friedman_mse','squared_error'],
              'max_depth':np.arange(1,15).tolist()[0::2],
              'min_samples_split':np.arange(2,11).tolist()[0::2],
              'max_leaf_nodes':np.arange(3,20).tolist()[0::2]}
tuning_model=GridSearchCV(decision_tree_model,parameters,scoring='neg_mean_squared_error',cv=10,verbose=3, n_jobs=1)

In [9]:
tuning_model.fit(X_train,y_train)

Fitting 10 folds for each of 630 candidates, totalling 6300 fits
[CV 1/10] END criterion=friedman_mse, max_depth=1, max_leaf_nodes=3, min_samples_split=2;, score=-100.374 total time=   0.3s
[CV 2/10] END criterion=friedman_mse, max_depth=1, max_leaf_nodes=3, min_samples_split=2;, score=-101.085 total time=   0.3s
[CV 3/10] END criterion=friedman_mse, max_depth=1, max_leaf_nodes=3, min_samples_split=2;, score=-97.340 total time=   0.2s
[CV 4/10] END criterion=friedman_mse, max_depth=1, max_leaf_nodes=3, min_samples_split=2;, score=-93.680 total time=   0.2s
[CV 5/10] END criterion=friedman_mse, max_depth=1, max_leaf_nodes=3, min_samples_split=2;, score=-94.112 total time=   0.2s
[CV 6/10] END criterion=friedman_mse, max_depth=1, max_leaf_nodes=3, min_samples_split=2;, score=-93.470 total time=   0.2s
[CV 7/10] END criterion=friedman_mse, max_depth=1, max_leaf_nodes=3, min_samples_split=2;, score=-91.913 total time=   0.3s
[CV 8/10] END criterion=friedman_mse, max_depth=1, max_leaf_nodes

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(), n_jobs=1,
             param_grid={'criterion': ['friedman_mse', 'squared_error'],
                         'max_depth': [1, 3, 5, 7, 9, 11, 13],
                         'max_leaf_nodes': [3, 5, 7, 9, 11, 13, 15, 17, 19],
                         'min_samples_split': [2, 4, 6, 8, 10]},
             scoring='neg_mean_squared_error', verbose=3)

In [13]:
tuning_model.best_params_

{'criterion': 'friedman_mse',
 'max_depth': 7,
 'max_leaf_nodes': 19,
 'min_samples_split': 2}

In [15]:
tuned_decision_tree_model = DecisionTreeRegressor(max_depth=7,max_leaf_nodes=19, min_samples_split=2, criterion='friedman_mse')
tuned_decision_tree_model.fit(X_train, y_train)

DecisionTreeRegressor(criterion='friedman_mse', max_depth=7, max_leaf_nodes=19)

In [17]:
# Show our starting off point for error
yhat_train = tuned_decision_tree_model.predict(X_train)
train_error = mean_squared_error(y_train,yhat_train)
training_error_list.append(np.log(train_error))
print('Training Error:',train_error)

yhat_test = tuned_decision_tree_model.predict(X_test)
test_error = mean_squared_error(y_test,yhat_test)
testing_error_list.append(np.log(test_error))
print('Testing Error:',test_error)

yhat_val = tuned_decision_tree_model.predict(X_val)
val_error = mean_squared_error(y_val,yhat_val)
validation_error_list.append(np.log(val_error))
print('Validation Error:',val_error)

Training Error: 91.1911685186354
Testing Error: 91.03768249546283
Validation Error: 88.58676438078393
